In [1]:
import torch
from torch import nn


class VariationalAutoEncoder(nn.Module):
    def __init__(self, input_dim, h_dim=200, z_dim=20):
        super().__init__()
        # encoder
        self.img_2hid = nn.Linear(input_dim, h_dim)
        self.hid_2mu = nn.Linear(h_dim, z_dim)
        self.hid_2sigma = nn.Linear(h_dim, z_dim)

        # decoder
        self.z_2hid = nn.Linear(z_dim, h_dim)
        self.hid_2img = nn.Linear(h_dim, input_dim)

        self.relu = nn.ReLU()

    def encode(self, x):
        h = self.relu(self.img_2hid(x))
        mu, sigma = self.hid_2mu(h), self.hid_2sigma(h)
        return mu, sigma

    def decode(self, z):
        h = self.relu(self.z_2hid(z))
        return torch.sigmoid(self.hid_2img(h))

    def forward(self, x):
        mu, sigma = self.encode(x)
        # reparametrization trick
        epsilon = torch.randn_like(sigma)
        z_new = mu + sigma*epsilon
        x_reconstructed = self.decode(z_new)
        return x_reconstructed, mu, sigma


In [3]:
import torch
import torchvision.datasets as datasets  # Standard datasets
from tqdm import tqdm
from torch import nn, optim
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_DIM = 784
H_DIM = 200
Z_DIM = 20
NUM_EPOCHS = 30
BATCH_SIZE = 32
LR_RATE = 3e-4  # Karpathy constant

# Dataset Loading
dataset = datasets.FashionMNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)
model = VariationalAutoEncoder(INPUT_DIM, H_DIM, Z_DIM).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR_RATE)
loss_fn = nn.BCELoss(reduction="sum")

# Training
for epoch in range(NUM_EPOCHS):
    loop = tqdm(enumerate(train_loader))
    for i, (x, _) in loop:
        # forward pass
        x = x.to(DEVICE).view(x.shape[0], INPUT_DIM)
        x_reconstructed, mu, sigma = model(x)

        # compute loss
        reconstruction_loss = loss_fn(x_reconstructed, x)
        kl_div = -torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2))

        # backprop
        loss = reconstruction_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())




100%|██████████| 26421880/26421880 [00:01<00:00, 16113881.18it/s]


Extracting dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to dataset/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 3736978.37it/s]


Extracting dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to dataset/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6330274.20it/s]


Extracting dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to dataset/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 23093344.38it/s]


Extracting dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/FashionMNIST/raw



1875it [00:08, 229.00it/s, loss=8.62e+3]
1875it [00:08, 228.77it/s, loss=8.12e+3]
1875it [00:08, 231.76it/s, loss=9.62e+3]
1875it [00:08, 230.30it/s, loss=7.8e+3] 
1875it [00:07, 240.73it/s, loss=8.29e+3]
1875it [00:07, 237.77it/s, loss=8.9e+3] 
1875it [00:07, 236.05it/s, loss=8.56e+3]
1875it [00:07, 242.13it/s, loss=8.56e+3]
1875it [00:07, 243.09it/s, loss=8.27e+3]
1875it [00:07, 243.57it/s, loss=8.63e+3]
1875it [00:07, 239.23it/s, loss=9.01e+3]
1875it [00:08, 232.20it/s, loss=7.89e+3]
1875it [00:08, 228.25it/s, loss=8.48e+3]
1875it [00:08, 228.90it/s, loss=9.15e+3]
1875it [00:07, 235.67it/s, loss=7.21e+3]
1875it [00:07, 240.76it/s, loss=8.43e+3]
1875it [00:07, 241.43it/s, loss=8.34e+3]
1875it [00:07, 241.01it/s, loss=7.99e+3]
1875it [00:07, 239.10it/s, loss=8.14e+3]
1875it [00:07, 237.73it/s, loss=8.13e+3]
1875it [00:07, 238.31it/s, loss=7.68e+3]
1875it [00:07, 238.08it/s, loss=8.04e+3]
1875it [00:07, 244.38it/s, loss=7.74e+3]
1875it [00:07, 241.65it/s, loss=7.63e+3]
1875it [00:07, 2

- T-shirt/top
- Trouser
- Pullover
- Dress
- Coat
- Sandal
- Shirt
- Sneaker
- Bag
- Ankle boot

In [9]:
def inference(label, num_examples=1):
    """
    Generates (num_examples) of a particular digit.
    Specifically we extract an example of each digit,
    then after we have the mu, sigma representation for
    each digit we can sample from that.

    After we sample we can run the decoder part of the VAE
    and generate examples.
    """
    for x, y in dataset:
        if y == label:
            images = x
            break
    
    mu, sigma = model.encode(images.view(1, 784))

    for example in range(num_examples):
        epsilon = torch.randn_like(sigma)
        z = mu + sigma * epsilon
        out = model.decode(z)
        out = out.view(-1, 1, 28, 28)
        save_image(out, f"output/FashionMNIST/generated_{label}_ex{example}.png")

for label in range(10):
    inference(label, num_examples = 4)